# What's this??
Seleniumを使ってATBBからデータを引っ張ってくる

http://atbb.athome.jp/

In [1]:
# import libraries
import time
import pandas as pd
import numpy as np
import os
import urllib.request

In [2]:
# import selenium libraries
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [3]:
# import original libraries
from utils.python import atbb

In [4]:
# variables
path = '/Users/ryusukekimura/Google ドライブ/github/demonstration/rpa/chrome-driver-79.0.3945.36/chromedriver'
url = 'http://atbb.athome.jp/'
timeout = 15

In [5]:
# create output directory
dir_data = './data/atbb/'
if not os.path.isdir(dir_data):
    os.makedirs(dir_data)
dir_img = './img/atbb/'
if not os.path.isdir(dir_img):
    os.makedirs(dir_img)

In [6]:
# define output columns
cols =  ['bld_name', 'room_name', 'room_madori', 'bld_address', 'access', 'num_rooms',
              'chinryo', 'kanrihi', 'sikikin', 'reikin', 'kibo_kozo', 'menseki_tsubo', 'syunko_nengappi', 'keiyaku_kikan', 'kousinryo',
               'torihiki_taiyo', 'parking', 'hoken', 'kanri_gaisya', 'kagi_koukan', 'kouza_furikae', 'hosyo_gaisya', 'setsubi', 'bikou']

In [7]:
# login
## open login page
driver = webdriver.Chrome(executable_path=path)
driver.get(url)

## read id and password
id, pw = atbb.read_id_pass()

## input id and password
driver.find_element_by_id('loginFormText').send_keys(id)
driver.find_element_by_id('passFormText').send_keys(pw)

## click login button
driver.find_element_by_xpath('//*[@id="loginSubmit"]').click()

## wait
time.sleep(timeout)

In [ ]:
# for each shikuchouson
for i in range(1, 24):
    print('|-- i = ' + str(i) + ' ...')
    df = pd.DataFrame(index=[], columns=[cols])
    driver = webdriver.Chrome(executable_path=path)
    driver.get(url)
    mitsui.move_to_search_page_from_top_page(driver, timeout)
    driver.find_element_by_xpath('//*[@id="(1,' + str(i) + ')"]').click()
    mitsui.move_to_search_result_page_from_search_page(driver, timeout)
    num_pages = int(driver.find_element_by_xpath('//*[@id="container"]/section/div[2]/div[2]/div[1]/span[1]').text)

    # for each page
    for page in range(1, num_pages + 1):
        print('  |-- processing page ' + str(page) + ' ...')

        for bukken in range(1, 11):
            print('     |-- bukken number ' + str(bukken) + ' ...')

            # extract room information
            driver.find_element_by_xpath('//*[@id="container"]/section/li[' + str(bukken) + ']/div[2]/div[1]/div/h3').click()
            time.sleep(timeout)
            array = mitsui.extract_information(driver)

            # add array to dataframe
            record = pd.Series(array, index=df.columns)
            df = df.append(record, ignore_index=True)

            # go back to search result pages
            driver.back()
            time.sleep(timeout)

        # move to next page
        if page != num_pages:
            driver.find_element_by_xpath('//*[@id="container"]/section/div[1]/div/div/div[2]/div[2]/div[4]/ul/li[2]/a').click()
            time.sleep(timeout)

    # output
    file = './data/mitsui/output' + str(i) + '.csv'
    df.to_csv(file, index=False)
            
    # close driver
    driver.close()